In [3]:
import os
import sys
import datetime
from time import gmtime
from time import strftime
from pathlib import Path
from datetime import timedelta  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import signal
import pickle
import librosa
import librosa.display
from feature_extraction import data_ls_to_string

from spectral_subtraction import spectral_subtraction

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
import zipfile
from zipfile import ZIP_LZMA
from zipfile import ZIP_DEFLATED

def zip_wav_files(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all files in the input directory
    for filename in os.listdir(input_dir):

        if filename.lower().endswith(".wav"):
            # Define the full file paths
            file_path = os.path.join(input_dir, filename)
            zip_file_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.zip")

            # Check if the zip file already exists
            if os.path.exists(zip_file_path):
                print(f"Skipping {filename}, already zipped.")
                continue

            # Create a zip file and add the wav file to it using ZIP_LZMA for maximum compression
            #with zipfile.ZipFile(zip_file_path, mode='x', compression=ZIP_LZMA) as zip_file:
            with zipfile.ZipFile(zip_file_path, mode='x', compression=ZIP_DEFLATED, compresslevel=9,
                                 strict_timestamps=False) as zip_file:
                zip_file.write(file_path, arcname=filename)

            print(f"Zipped {filename} to {zip_file_path}")

# Define the input and output directories
input_directory = '../data/Nectar/full_2021'
output_directory = '../data/Nectar/full_2021_zipped'

# Zip the wav files
zip_wav_files(input_directory, output_directory)

In [ ]:
import os
import shutil

def move_hive_0_files(source_dir, target_dir):
    # Create the target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    # Iterate over all files in the source directory
    for filename in os.listdir(source_dir):
        if "HIVE-0" in filename:
            # Construct full file path
            file_path = os.path.join(source_dir, filename)
            # Move the file to the target directory
            shutil.move(file_path, target_dir)
            print(f"Moved: {filename}")

# Define your source and target directories
source_directory = '../data/Nectar/full_2022'
target_directory = '../data/Nectar/empty_2022'

# Call the function to move files
move_hive_0_files(source_directory, target_directory)


In [ ]:
folder_path = "../data/Nectar/full_2021"

# Initialize total size
total_size = 0
num = 0

# Loop through files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a WAV file and has tag == 0
    filename = filename[:-4] + ".wav"
    if filename.endswith(".wav") and filename.split("_")[2].split("-")[1] != "0.wav":
        # Extract date from the filename
        date_str = filename.split("_")[0]
        time_str = filename.split("_")[1]
        datetime_str = date_str + "_" + time_str
        
        datetime_obj = datetime.datetime.strptime(datetime_str, "%d-%m-%Y_%Hh%M")

        
        if datetime_obj >= datetime.datetime(2021,1, 1) and datetime_obj <= datetime.datetime(2021, 12, 31):
        #if datetime_obj >= datetime.datetime(2021,6, 22) and datetime_obj <= datetime.datetime(2021, 10, 19):
            # Get the file size
            # Get the file size
            file_size = os.path.getsize(os.path.join(folder_path, filename))
            # Add to total size
            total_size += file_size
            num += 1
            
total_size_gb = total_size / (1024 ** 3)
total_size_zipped_gb = (num*18) / (1024 ** 1)


# Print the total size
print("Total size of files:", total_size_gb, "GB")
print("Total size of zipped files:", total_size_zipped_gb, "GB")

print("Total number of files:", num )

total_time = num*15

h=total_time//60
m=total_time%60
print("Hours=",h)
print("Minutes=",m)

In [ ]:
folder_path = "../data/Nectar/full_2022"

# Initialize total size
total_size = 0
num = 0


# Loop through files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a WAV file and has tag == 0
    filename = filename[:-4] + ".wav"
    if filename.endswith(".wav") and filename.split("_")[2].split("-")[1] != "0.wav":
        # Extract date from the filename
        date_str = filename.split("_")[0]
        time_str = filename.split("_")[1]
        datetime_str = date_str + "_" + time_str
        datetime_obj = datetime.datetime.strptime(datetime_str, "%d-%m-%Y_%Hh%M")
        # Check if the date is between June and August 2021
        if datetime_obj >= datetime.datetime(2022,1, 1) and datetime_obj <= datetime.datetime(2022, 12,31):
        #if datetime_obj >= datetime.datetime(2022,7, 11) and datetime_obj <= datetime.datetime(2022, 9, 7):
            # Get the file size
            file_size = os.path.getsize(os.path.join(folder_path, filename))
            # Add to total size
            total_size += file_size
            num += 1
            
total_size_gb = total_size / (1024 ** 3)
total_size_zipped_gb = (num*18) / (1024 ** 1)


# Print the total size
print("Total size of files:", total_size_gb, "GB")
print("Total size of zipped files:", total_size_zipped_gb, "GB")

print("Total number of files:", num )

total_time = num*15

h=total_time//60
m=total_time%60
print("Hours=",h)
print("Minutes=",m)




# 2021 data

In [ ]:
data = pd.read_csv("../data/annotations/inspections_2021.csv")
data

In [ ]:
# dead hives : 3640, 3629
#after winter: 3627, 3628, 3629, 3692

In [ ]:
plt.rc('legend',fontsize=14) # using a size in points
plt.rcParams['xtick.labelsize'] = 20 
plt.rcParams['ytick.labelsize'] = 20 

fig, ax = plt.subplots(figsize=(16,12), sharex=True)
sns.histplot(data["Fob 1st"]+data["Fob 2nd"]+data["Fob 3rd"], bins = np.arange(20,32)-0.5, color='steelblue')
plt.xticks(np.arange(20,31));
plt.xlabel('Frames of bee', size = 22)
plt.ylabel('Count', size = 22)
plt.savefig('fob_2021.png', dpi=400)

In [ ]:
notes_and_opening = data[['Date', 'Tag number', 'Open', 'Close', 'Notes']]
notes_and_opening.dropna(subset = ['Date', 'Open'], inplace=True)
notes_and_opening['Date'] = pd.to_datetime(notes_and_opening['Date'], dayfirst=True)
notes_and_opening.reset_index(inplace=True)
notes_and_opening.info()

In [ ]:
        
notes_and_opening['Open'] = pd.to_datetime(notes_and_opening['Open'], dayfirst=True)
notes_and_opening['Close'] = pd.to_datetime(notes_and_opening['Close'], dayfirst=True)

for i in range(len(notes_and_opening)):
    if not pd.isna(notes_and_opening.Close[i]):
        notes_and_opening['Close'][i] = datetime.datetime.combine(notes_and_opening.Date[i], notes_and_opening.Close[i])
        #notes_and_opening['Close'] = pd.to_datetime(notes_and_opening['Close'], dayfirst=True)
        
for i in range(len(notes_and_opening)):
    if not pd.isna(notes_and_opening.Open[i]):
        notes_and_opening['Open'][i] = datetime.datetime.combine(notes_and_opening.Date[i], notes_and_opening.Open[i])
        #notes_and_opening['Close'] = pd.to_datetime(notes_and_opening['Close'], dayfirst=True)
        

notes_and_opening.drop('index', inplace=True, axis=1)

In [ ]:
data = data.fillna(0)
data['Date'] = pd.to_datetime(data['Date']).dt.strftime("%Y-%m-%d")
#Enotes_and_opening['Tag'] = notes_and_opening['Tag'].apply(lambda x: str(x).replace(u'\xa0', u'')).astype('int')

In [ ]:
data['fob'] = data["Fob 1st"] + data["Fob 2nd"]+ data["Fob 3rd"]
data= data.set_index(data['Date'])

In [ ]:
plt.rc('legend',fontsize=10) # using a size in points
plt.rcParams['xtick.labelsize'] = 20 
plt.rcParams['ytick.labelsize'] = 20 

fig, ax = plt.subplots(figsize=(16,6), sharex=True)

sns.barplot(data, x=data.index, y='fob', hue="Tag number")
plt.setp(ax.get_legend().get_title(), fontsize='12') # for legend title
ax.legend(loc='upper left',ncol=2, title="Hive ID")

plt.xticks(rotation=45)
plt.ylabel('Frames of bee', size = 22)
plt.xlabel('Date', size = 22)
plt.tight_layout()
plt.savefig('fob_detailed_2021.png', dpi=400)


In [ ]:
unique_hives = data['Tag number'].unique()
unique_hives

In [ ]:
grouped = data.groupby(['Tag number'])
dict_hives = {}
for i in unique_hives:
        dict_hives[i] = grouped.get_group(i)

In [ ]:
for hive in  data['Tag number'].unique():
    dict_hives[hive] = dict_hives[hive].set_index(dict_hives[hive]['Date'])
    idx = pd.date_range(dict_hives[hive].index.min(), dict_hives[hive].index.max()+ timedelta(days=1), freq="15min")#  + timedelta(days=12)
    dict_hives[hive] = dict_hives[hive].reindex(idx)
    dict_hives[hive] = dict_hives[hive].drop(['Date'], axis=1)
    dict_hives[hive] = dict_hives[hive].interpolate(method="linear")#interpolate(method="ffill")
    dict_hives[hive]["fob"] = dict_hives[hive]["Fob 1st"] + dict_hives[hive]["Fob 2nd"]+ dict_hives[hive]["Fob 3rd"]
    dict_hives[hive]["fob"] = dict_hives[hive]["fob"].round(0).astype('f')

In [ ]:
dict_hives[3631]

# read csv file 2022

In [ ]:
data_2022 = pd.read_csv("../data/annotations/inspections_2022.csv")
data_2022['Date'] = pd.to_datetime(data_2022['Date'], dayfirst=True).dt.date
data_2022 = data_2022.set_index('Date')
data_2022['Tag number'] = data_2022['Tag number'].astype('int')

data_2022

# Varroa measurement

In [ ]:
df_varroa = data_2022[data_2022['Category'] == 'varroa'][['Action detail', 'Tag number']]
df_varroa['Action detail'] = df_varroa['Action detail'].astype('f')
df_varroa['Tag number'] = df_varroa['Tag number'] .astype('int')

df_varroa.index = pd.to_datetime(df_varroa.index, dayfirst=True)
df_varroa['date'] = df_varroa.index
df_varroa['date'] = df_varroa['date'].dt.date

In [ ]:
#df_varroa_temp
plt.rc('legend',fontsize=16) # using a size in points
plt.rcParams['xtick.labelsize'] = 16 
plt.rcParams['ytick.labelsize'] = 16 


for d in df_varroa.index.unique():
    
    df_varroa_temp = df_varroa.loc[d]

    df_varroa_temp = df_varroa_temp.sort_values('Tag number')
    fig, ax = plt.subplots(figsize=(12, 10))

    ax = sns.barplot(df_varroa_temp.sort_values('Tag number'), x="Tag number", y='Action detail', color='steelblue')

    for i, p in enumerate(ax.patches):
        ax.text(p.get_x() + p.get_width() / 2., p.get_height(), df_varroa_temp['Action detail'][i],
                ha='center', va='bottom', color='black', fontsize=16, weight='bold');

    plt.xlabel('Hive ID', size = 18)
    plt.ylabel('Varroa infestation', size = 18)
    #plt.title('Varroa infestation measurement on ' + str(d)[:10] , weight = 'bold')
    #plt.savefig('varroa' +  str(d)   + '.png', dpi=400) 


In [ ]:
# dead hives : 3628, 3629, 3631


# Frames of bees

In [ ]:
data_2022 = data_2022[data_2022['Category'] == 'frames of bees']
data_2022['Action detail'] = pd.to_numeric(data_2022['Action detail']) # conver the column from object to float
data_2022['Action detail'] = data_2022['Action detail'].astype('f') # covert from float to int

In [ ]:
data_2022["fob"] = data_2022["Action detail"]

In [ ]:
plt.rc('legend',fontsize=10) # using a size in points
plt.rcParams['xtick.labelsize'] = 20 
plt.rcParams['ytick.labelsize'] = 20 

fig, ax = plt.subplots(figsize=(16,6), sharex=True)

sns.barplot(data_2022, x=data_2022.index, y='fob', hue="Tag number")
plt.setp(ax.get_legend().get_title(), fontsize='12') # for legend title
ax.legend(loc='upper right',ncol=2, title="Hive ID")

plt.xticks(rotation=45)
plt.ylabel('Frames of bee', size = 22)
plt.xlabel('Date', size = 22)
plt.tight_layout()
plt.savefig('fob_detailed_2022.png', dpi=400)


In [ ]:
plt.rc('legend',fontsize=14) # using a size in points
plt.rcParams['xtick.labelsize'] = 20 
plt.rcParams['ytick.labelsize'] = 20 

fig, ax = plt.subplots(figsize=(16,12), sharex=True)
sns.histplot(data_2022["fob"] , bins = np.arange(1,22)-0.5, color='steelblue')
plt.xticks(np.arange(1,21));
plt.xlabel('Frames of bee', size = 22)
plt.ylabel('Count', size = 22)

plt.savefig('fob_2022.png', dpi=400)

In [ ]:
grouped = data_2022.groupby(['Tag number'])
dict_hives_2022 = {}
for i in data_2022['Tag number'].unique():
        dict_hives_2022[i] = grouped.get_group(i)

In [ ]:
for hive in  data_2022['Tag number'].unique():
    idx = pd.date_range(dict_hives_2022[hive].index.min(), dict_hives_2022[hive].index.max(), freq="15min")
    dict_hives_2022[hive] = dict_hives_2022[hive].reindex(idx)
    dict_hives_2022[hive] = dict_hives_2022[hive].interpolate(method="linear")#interpolate(method="ffill")
    dict_hives_2022[hive]['fob'] = dict_hives_2022[hive]['fob'].round(0).astype('f')

In [ ]:
dict_hives_2022[3631]

# Temperature and humidiity 

In [ ]:
full_sensor_df= pd.read_csv("../data/temperature_humidity/sensor_2021.csv")
full_sensor_df

In [ ]:
grouped = full_sensor_df.groupby(['Tag number'])
sensors_hives = {}
for i in full_sensor_df['Tag number'].unique():
        sensors_hives[i] = grouped.get_group(i)

In [ ]:
for hive in  full_sensor_df['Tag number'].unique():
    sensors_hives[hive]['Date'] = pd.to_datetime(sensors_hives[hive]['Date'])

    sensors_hives[hive] = sensors_hives[hive].set_index(sensors_hives[hive]['Date'])
    sensors_hives[hive] = sensors_hives[hive].drop(['Date'], axis=1)

In [ ]:
resampled_sensor_hive = {}
x = {}
for hive in full_sensor_df['Tag number'].unique():
    x[hive] = sensors_hives[hive].resample('h').mean()
    idx = pd.date_range(x[hive].index.min(), x[hive].index.max() + timedelta(days=1) - 
                        pd.to_timedelta(x[hive].index.max().hour, unit='h')
                        - pd.to_timedelta(1, unit='h'), freq='H')
    x[hive] = x[hive].reindex(idx).interpolate(method="linear") #interpolate(method="spline", order=2, limit = 100, limit_direction='both')
    #x[hive] = x[hive].dropna(subset=['audio_density'])
    if len(x[hive]) > 0:
        resampled_sensor_hive[hive] = x[hive]

# Weather info

In [ ]:
env = pd.read_csv("../data/weather_info/weather_2021_2022.csv")
env['Date/Time (LST)'] = pd.to_datetime(env['Date/Time (LST)'])

env = env.set_index(env['Date/Time (LST)'])
env = env.interpolate(method="ffill")

In [ ]:
df = full_sensor_df

df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index(df['Date'])
df = df.drop(['Date'], axis=1)
#df = df.interpolate(method="ffill")
df = df.resample('h').mean()

df = df.groupby(df.index).mean()
idx = pd.date_range(df.index.min(), df.index.max() + timedelta(days=1) - 
                    pd.to_timedelta(df.index.max().hour, unit='h')
                    - pd.to_timedelta(1, unit='h'), freq='H')
df = df.reindex(idx).interpolate(method="linear")

In [ ]:
plt.rc('legend',fontsize=18) # using a size in points
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14 


fig, ax = plt.subplots(figsize=(16,10))


plt.subplot(2, 1,1)
plt.plot(df['temperature'].loc['2021-06-15':'2021-10-30'].resample('d').mean(), color='red', linestyle='dashed',
linewidth=4, markersize=12, label = "Internal")
plt.plot(env['Temp (°C)'].loc['2021-06-15':'2021-10-30'].resample('d').mean(), color='orange', linestyle='dashed',
linewidth=4, markersize=12, label = "External")


#plt.xticks(sensors_hives[hive_id.hive_identity[hive]].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'temperature'].resample('30min').mean().index[0:48:4], idx_12.strftime('%H:%M'))
#plt.xlim([sensors_hives[hive_id.hive_identity[hive]].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'temperature'].resample('30min').mean().index.min(), sensors_hives[hive_id.hive_identity[hive]].loc['2021-8-12 00:00':'2021-8-13 00:00', 'temperature'].resample('15min').mean().index.max()])
plt.ylabel('Temperature ($^\circ$C)', size=24)
plt.legend()


plt.subplot(2, 1, 2)
plt.plot(df['humidity'].loc['2021-06-15':'2021-10-30'].resample('d').mean(), color='blue', linestyle='dashed',
linewidth=4, markersize=12, label = "Internal ")
plt.plot(env['Rel Hum (%)'].loc['2021-06-15':'2021-10-30'].resample('d').mean(), color='green', linestyle='dashed',
linewidth=4, markersize=12, label = "External")



#plt.xticks(sensors_hives[hive_id.hive_identity[hive]].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'humidity'].resample('30min').mean().index[0:48:4], idx_12.strftime('%H:%M'))
#plt.xlim([sensors_hives[hive_id.hive_identity[hive]].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'humidity'].resample('30min').mean().index.min(), sensors_hives[hive_id.hive_identity[hive]].loc['2021-8-12 00:00':'2021-8-13 00:00', 'temperature'].resample('15min').mean().index.max()])
plt.ylabel('Humidity (%)', size=24)
plt.xlabel('Time', size=24)

plt.legend()

plt.tight_layout()
plt.savefig('temp_humid_total_2021.png')
#plt.close('all')

# Synchronizing empty and beehive

In [ ]:
date = '2021-08-16'
hive = 3691

idx = pd.date_range(date, periods=96, freq="15min")
df_time = pd.DataFrame(data=np.array(np.zeros(96)), index = idx, columns=['Tag'], dtype=None, copy=None)
df_time["Tag"] = df_time["Tag"].astype('int')

file_names_empty = data_index_to_string(df_time)
file_names_bee = data_index_to_string(dict_hives[hive].loc[date])

ls_dates = []
empty = np.array(1)
bee = np.array(1)

for file in range(len(file_names_bee)):
    try:
        signal_bee, sample_rate = librosa.load(os.path.join('../data/Nectar/full/', file_names_bee[file]), sr=2048)
        print(file_names_bee[file])
        signal_empty, sample_rate = librosa.load(os.path.join('../data/Nectar/full/', file_names_empty[file]), sr=2048)
        print(file_names_empty[file])
        bee = np.concatenate((bee, signal_bee[:min(signal_bee.shape[0], signal_empty.shape[0])]), axis=None)
        ls_dates.append(dict_hives[hive].loc[date].index[file])
        print(bee.shape)
        
        empty = np.concatenate((empty, signal_empty[:min(signal_bee.shape[0], signal_empty.shape[0])]), axis=None)
        print(empty.shape)
        
    except OSError as e:
            pass#print('The file is not present.') 
        
np.save(str(hive) + '_' + str(date) + '_12h.npy', bee)
np.save(str(0) + '_' + str(hive) + '_' + str(date) + '_12h.npy', empty)
np.save(str(hive) + '_' + str(date) + '_times.npy', ls_dates)

In [ ]:
plt.rc('legend',fontsize=18) # using a size in points
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14 



date = '2021-08-12'
hive = 3631

bee_12h = np.load(str(hive) + '_' + str(date) + '_12h.npy')

empty_hive_12h = np.load(str(0) + '_' + str(hive) + '_' + str(date) + '_12h.npy')

ls_dates  = np.load(str(hive) + '_' + str(date) + '_times.npy', allow_pickle=True)
idx_12 = pd.date_range(ls_dates[0], periods=12, freq="2H")

fig, ax = plt.subplots(figsize=(16,10))


plt.subplot(2, 1,1)
plt.plot(sensors_hives[hive].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1),
                                 'temperature'].resample('30min').mean().interpolate(method="linear"), color='red', linestyle='dashed',
     linewidth=4, markersize=12, label = "Internal")
plt.plot(env['Temp (°C)'][date], color='orange', linestyle='dashed',
     linewidth=4, markersize=12, label = "External")


plt.xticks(sensors_hives[hive].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'temperature'].resample('30min').mean().index[0:48:4], idx_12.strftime('%H:%M'))
plt.xlim([sensors_hives[hive].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'temperature'].resample('30min').mean().index.min(), 
          sensors_hives[hive].loc['2021-8-12 00:00':'2021-8-13 00:00', 'temperature'].resample('15min').mean().index.max()])
plt.ylabel('Temperature ($^\circ$C)', size=24)
plt.legend()


plt.subplot(2, 1, 2)
plt.plot(sensors_hives[hive].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'humidity'].resample('30min').mean().interpolate(method="linear"), color='blue', linestyle='dashed',
     linewidth=4, markersize=12, label = "Internal ")
plt.plot(env['Rel Hum (%)'][date], color='green', linestyle='dashed',
     linewidth=4, markersize=12, label = "External")

plt.xticks(sensors_hives[hive].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'humidity'].resample('30min').mean().index[0:48:4], idx_12.strftime('%H:%M'))
plt.xlim([sensors_hives[hive].loc[pd.to_datetime(date):pd.to_datetime(date)+ timedelta(days=1), 'humidity'].resample('30min').mean().index.min(),
          sensors_hives[hive].loc['2021-8-12 00:00':'2021-8-13 00:00', 'temperature'].resample('15min').mean().index.max()])
plt.ylabel('Humidity (%)', size=24)
plt.xlabel('Time', size=24)

plt.legend()

plt.tight_layout()
plt.savefig('paper_temp_humid_12h_'+ str(hive) + '_' + str(date) +'.png')
#plt.close('all')

In [ ]:
plt.rcParams['agg.path.chunksize'] = 10000

plt.rc('legend',fontsize=16) # using a size in points
plt.rcParams['xtick.labelsize'] = 14 
plt.rcParams['ytick.labelsize'] = 14 

date = '2021-08-16'
hive = 3691

bee_12h = np.load(str(hive) + '_' + str(date) + '_12h.npy')
ls_dates  = np.load(str(hive) + '_' + str(date) + '_times.npy', allow_pickle=True)

rolling = 1000

rms=librosa.feature.rms(y=bee_12h, S=None, frame_length=2048, hop_length=512, center=True, pad_mode='constant')

ma_rms = pd.Series(rms.reshape(-1)).rolling(rolling).mean()

idx_12 = pd.date_range(ls_dates[0], periods=12, freq="2H")
min15_size = int((ma_rms.shape[0] - rolling + 1)/len(ls_dates)) #48
min15_size_raw = int(bee_12h.shape[0]/len(ls_dates)) #48

fig = plt.figure()
fig.set_figheight(10)
fig.set_figwidth(16)


ax3 = plt.subplot2grid(shape=(3, 5), loc=(0, 0), colspan=4)

ax1 = plt.subplot2grid(shape=(3, 5), loc=(1, 0), colspan=4)
ax2 = plt.subplot2grid(shape=(3, 5), loc=(2, 0), colspan=5)

ax3.plot(bee_12h)
ax3.set_ylabel('Intensity', size=14)#Amplitude
ax3.set_xticks(np.arange(len(ls_dates)*min15_size_raw)[0:len(ls_dates)*min15_size_raw:4*min15_size_raw])
ax3.set_xticklabels(idx_12.strftime('%H:%M'))
ax3.axis(xmin=0,xmax=bee_12h.shape[-1])

ax1.semilogy(ma_rms.T[rolling:], label='Beehive')
ax1.set_xticks(np.arange(len(ls_dates)*min15_size)[0:len(ls_dates)*min15_size:4*min15_size]+rolling)
ax1.set_xticklabels(idx_12.strftime('%H:%M'))

ax1.set_ylabel('RMS', size=14)
ax1.set_xticks(np.arange(len(ls_dates)*min15_size)[0:len(ls_dates)*min15_size:4*min15_size]+rolling)
ax1.set_xticklabels(idx_12.strftime('%H:%M'))

ax1.axis(xmin=rolling,xmax=ma_rms.shape[0])


spec=librosa.display.specshow(librosa.amplitude_to_db(librosa.stft(bee_12h, n_fft=512), ref=np.max), sr=2048,  y_axis='log', cmap='jet')
plt.colorbar(spec, format="%+2.0f dB")
min15_size = int((librosa.stft(bee_12h, n_fft=512).shape[1])/len(ls_dates)) #48

ax2.set_xticks(np.arange(len(ls_dates)*min15_size)[0:len(ls_dates)*min15_size:4*min15_size]+rolling,)
ax2.set_xticklabels(idx_12.strftime('%H:%M'))

ax2.set_ylabel('Frequency (Hz)', size=14)
ax2.set_xlabel('Time', size=14)

plt.tight_layout()
plt.savefig('paper_spect_bee_'+ str(hive) + '_' + str(date) +'_' + 'rolling_' + str(rolling) +'.png')


# Raw audio and rms

In [ ]:
win = 1600
shift= 800


rms_2021 = pd.read_pickle("../data/features/2021_df_raw_rms_win_" + str(win) +'_shift_' + str(shift) +"_pkl")
rms_2022 = pd.read_pickle("../data/features/2022_df_raw_rms_win_" + str(win) +'_shift_' + str(shift) +"_pkl")

rms_2021['date'] = pd.to_datetime(rms_2021['date'], dayfirst=True)
rms_2021 = rms_2021.set_index(rms_2021['date'])
rms_2021 = rms_2021.drop(['date'], axis=1)

rms_2022['date'] = pd.to_datetime(rms_2022['date'], dayfirst=True)
rms_2022 = rms_2022.set_index(rms_2022['date'])
rms_2022 = rms_2022.drop(['date'], axis=1)

In [ ]:
df_all = rms_2021

df_24h = pd.DataFrame(columns=df_all.columns)

for hive in df_all["tag"].unique():

        print(hive)

        df = df_all[df_all["tag"]==hive].resample('h').mean()
        df = df.dropna(how='all')

        for date in np.unique(df.index.date):

            temp_date = df.loc[str(date)]
            #temp = temp.between_time(start, stop)
            if len(temp_date) == 24:
                #print(len(temp_date))
                print(date)
                print(len(temp_date))
                
                #df_24h = df_24h.drop(df_24h[(df_24h['tag'] == hive) & (df_24h.index.isin(temp_date.index))].index)
                df_24h = df_24h.append(df.loc[str(date)])
                #print(len(df_24h))

In [ ]:
for hive in df_24h["tag"].unique():

        print(hive)
        #df = df_all[df_all["tag"]==hive].resample('h').mean()
        #df = df.dropna(how='all')
        df = df_24h[df_24h["tag"]==hive]
        for date in np.unique(df.index.date):

            temp_date = df.loc[str(date)]
            #temp = temp.between_time(start, stop)
            if len(temp_date) < 24:
                #print(len(temp_date))
                print(date)
                

In [ ]:
plt.rc('legend',fontsize=16) # using a size in points
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

idx = [[10, 20], [20, 30], [1, 30]]

index_mutual=df_24h[(10 < df_24h["fob"]) &  (df_24h["fob"] <= 20)].index.intersection(df_24h[(20 < df_24h["fob"]) &  (df_24h["fob"] <= 30)].index)

for j in range(len(idx)):
    
    df_all_masked=df_24h[(idx[j][0] < df_24h["fob"]) &  (df_24h["fob"] <= idx[j][1])].loc[index_mutual]
    print(df_all_masked['tag'].unique())
    print(len(df_all_masked.index.date))
    print(set(df_all_masked.index.unique().date))
    print(len(df_all_masked.index.date)/df_all_masked['tag'].nunique()/24)
    
    #df_all_masked = df_all_masked.resample('h').mean()
    fig, ax = plt.subplots(figsize=(12, 10))

    res = df_all_masked.groupby(df_all_masked.index.hour).mean()
    sns.barplot(data=res, x=res.index,  y='rms', color='steelblue')
    plt.legend()
    plt.xlabel('Time (hour)', size=22)
    plt.ylabel('RMS', size=22)
    plt.savefig('Average rms of bee hives_' + str(idx[j][0]) + "_" + str(idx[j][1])  + '_same_dates.png', dpi=400) 


# Spectral subtraction

In [ ]:
sample_rate=16000

empty_name = "12-08-2021_00h00_HIVE-0.wav"
hive_name = "12-08-2021_00h00_HIVE-3631.wav"
signal_empty, sample_rate = librosa.load(os.path.join("../data/Nectar/full_2021/", empty_name), sr=sample_rate)
signal_bee, sample_rate = librosa.load(os.path.join("../data/Nectar/full_2021/", hive_name), sr=sample_rate)
shift = -1289884
signal_empty_realigned = shift_signal(signal_empty, 1289884)[1289884:]
signal_bee_realigned = signal_bee[1289884:]


In [ ]:
win_t = 30e-3#30e-3 # window size in seconds
x_hat_mag = spectral_subtraction(signal_bee_realigned, snr_threshold=1.5, window_time=30e-3, sample_rate = sample_rate)

plt.rc('legend',fontsize=18) # using a size in points
plt.rcParams['xtick.labelsize'] = 16 
plt.rcParams['ytick.labelsize'] = 16 

sample_rate = 16000
snr_threshold = 1.5 

bins = np.arange(len(signal_bee_realigned))[0:(len(signal_bee_realigned)):16000*120]
xlabels = ["0", "2:00", "4:00", "6:00", "8:00", "10:00", "12:00", "14:00"]

fig, ax = plt.subplots(2, 1, figsize=(16,10), sharex=True)
#fig.suptitle('Spectral subtraction', weight='bold', size=14)
plt.ylim(-1,1)

plt.legend()
plt.subplot(2, 1, 1)
plt.plot(signal_bee_realigned, label='Noisy signal', color='royalblue')
plt.ylim(-1,1)
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(x_hat_mag, label='Filtered signal', color='cornflowerblue')
plt.ylim(-1,1)

plt.legend()
plt.xticks(bins, xlabels[:len(bins)])
plt.xlabel('Time (minute)', size=16)
fig.supylabel('Amplitude', size=16)
plt.savefig('ss_amp_filtered_' + hive_name[:-4] +'_snr_' + str(snr_threshold) + '.png', dpi=400)
